# 一、分组和聚合

In [1]:
#1.简单例子

import pandas as pd
import numpy as np
data = pd.DataFrame({"company":['百度', '阿里', '百度', '阿里', '百度', '腾讯', '腾讯', '阿里', '腾讯', '阿里'],
                     'level': ['P7', 'P7', 'P8', 'P5', 'P8', 'P7', 'P8', 'P7', 'P5', 'P6'],
                     "salary":[43000, 24000, 40000, 39000, 8000, 47000, 25000, 16000, 21000, 38000],
                     "age":[25, 34, 49, 42, 28, 23, 45, 21, 34, 29]})

data

,company,level,salary,age
0,百度,P7,43000,25
1,阿里,P7,24000,34
2,百度,P8,40000,49
3,阿里,P5,39000,42
4,百度,P8,8000,28
5,腾讯,P7,47000,23
6,腾讯,P8,25000,45
7,阿里,P7,16000,21
8,腾讯,P5,21000,34
9,阿里,P6,38000,29


In [2]:
data.agg(['max','mean'])

,company,level,salary,age
max,阿里,P8,47000.0,49.0
mean,NaN,NaN,30100.0,33.0


In [3]:
# agg聚合函数，常与groupby联用，也可以直接与数据框联用
# 整个数据框所有列的最大值，无聚合则结果为nan

data.agg(['max','mean'])

# #年龄的平均值
data['age'].agg('mean')  #等价： data['age'].mean()  


# #求工资列的最小值、最大值以及中位数
data['salary'].agg(['min','max','median'])
#单列聚合，结果为系列


# 对不同字段聚合不同值,由于聚合结果也为数据框，所以有字段为空
# agg中用字典表示对不同列进行不同聚合
data.agg({'level': ['max','min'], 'salary': ['mean','std']})

# #上面结果也是数据框


,level,salary
max,P8,NaN
min,P5,NaN
mean,NaN,30100.000000
std,NaN,13016.655997


In [4]:
#打印每个公司工资的平均值
print('+'*50)
#说明，不用agg，直接用带有聚合性质的函数也可以达到聚合效果
print(data.groupby('company')['salary'].mean())   #结果为系列

print(data.groupby('company').agg({'salary':'mean'}))  #结果为数据框

#以上两句等价，但更推荐第二句，第二句执行出来的结果是数据框


++++++++++++++++++++++++++++++++++++++++++++++++++
company
百度    30333.333333
腾讯    31000.000000
阿里    29250.000000
Name: salary, dtype: float64
               salary
company              
百度       30333.333333
腾讯       31000.000000
阿里       29250.000000


In [5]:
#分组后，如果没有指明聚合字段，则所有数值型字段均聚合
#下例中，对工资、年龄分别求平均与最大值
data.groupby('company').agg(['mean','max'])



salary          age    
                 mean    max  mean max
company                               
百度       30333.333333  43000  34.0  49
腾讯       31000.000000  47000  34.0  45
阿里       29250.000000  39000  31.5  42

In [6]:
# 思考题：按公司分组后，求每个公司人员，年龄的均方根，并用算数方法验证
# 均方根 （RMS）公式：  sqrt( sum(x**2)/n )

#例如 计算百度公司人员年龄的均方根
#1. 列出百度公司人员情况，计算人数

indx=data['company']=='百度'
n=data[indx]['age'].shape[0]  #  n 为人员个数
data[indx]

,company,level,salary,age
0,百度,P7,43000,25
2,百度,P8,40000,49
4,百度,P8,8000,28


In [7]:
# 2.算数方法算出age的均方根：
import numpy as np
np.sqrt(  (25**2+49**2+28**2)/3  )
#               =sqrt((625+2401+784)/3)
#                      =sqrt(3810/3)=sqrt(1270)=35.637059

35.63705936241092

In [8]:
#3.由于numpy和pandas中没有直接计算均方根的函数，所以这个函数必须自己定义

rms =  lambda x :np.sqrt( sum(x**2)/n )

data.groupby('company').agg({'age': rms })  #所有公司内部age值相加后再平方

,age
company,
百度,35.637059
腾讯,35.166272
阿里,37.425482


In [9]:
#按公司分组后，求工资的均方根 （RMS）  sqrt( sum(x**2)/n )

data.groupby('level').agg({'salary': rms })  #所有公司内部age值相加后再平方

,salary
level,
P5,25573.423705
P6,21939.310229
P7,40373.258476
P8,27622.454634


# 二、多级分组

In [10]:
#多级分组中，不要把顺序写错
#求每个公司-每个层级工资的最大值、最小值、中位数
data.groupby(['company','level']).agg({'salary':['min','max','median']})
# print('-'*50)
# data.groupby(['company','level']).agg({'salary':['min','max','median']}).columns

salary              
                 min    max median
company level                     
百度      P7     43000  43000  43000
        P8      8000  40000  24000
腾讯      P5     21000  21000  21000
        P7     47000  47000  47000
        P8     25000  25000  25000
阿里      P5     39000  39000  39000
        P6     38000  38000  38000
        P7     16000  24000  20000

# 三、 综合练习：分组聚合

In [11]:
# 读入文件  meal_order_info.csv
# 1. 按天分组（星期），求消费的笔数，金额的最大值，最小值，金额总量
# 2. 求每周的人均消费信息
# 3. 按星期名称排序
# 本题不需要所有数据，因此值读出相应的列即可，usecols=[0,2,6,11]
# 分别为：info_id，number_consumers，expenditure，lock_time四列

import pandas as pd
import numpy as np

# 读csv文件，默认utf-8
order=pd.read_csv("d:/aa/meal_order_info.csv",encoding="gbk",usecols=[0,2,6,11])
order

FileNotFoundError: [Errno 2] No such file or directory: 'd:/aa/meal_order_info.csv'

In [2]:
# order['lock_time']列改为时间类型
order['lock_time']=pd.to_datetime(order['lock_time'])  


# order['date']=order['lock_time'].dt.date   # 增加date列，去掉时间中的时分秒
order['week']=order['lock_time'].dt.isocalendar().week   
order

,info_id,number_consumers,expenditure,lock_time,week
0,417,4,165,2016-08-01 11:11:46,31
1,301,3,321,2016-08-01 11:31:55,31
2,413,6,854,2016-08-01 12:54:37,31
3,415,4,466,2016-08-01 13:08:20,31
4,392,10,704,2016-08-01 13:07:16,31
...,...,...,...,...,...
940,641,8,679,2016-08-31 21:31:48,35
941,672,6,800,2016-08-31 21:56:12,35
942,692,8,735,2016-08-31 21:33:34,35
943,647,4,262,2016-08-31 21:55:39,35


In [3]:
# 按日期分组统计，订单数，消费人数，营业额的总金额，均值（此时均值实际上就是总金额除以订单数）
# order31=order.groupby('date').agg({"info_id":'count','number_consumers':'sum',"expenditure":['sum','mean']})
# order31.head()

In [17]:
#按周统计
order['week']=order['lock_time'].dt.isocalendar().week   
# order['week']=order['lock_time'].dt.week #增加week列
order0=order.groupby('week').agg({"info_id":'count','number_consumers':'sum',"expenditure":['sum','mean']})
order0

info_id number_consumers expenditure            
       count              sum         sum        mean
week                                                 
31       223             1157      103218  462.860987
32       212             1096      103185  486.721698
33       237             1261      123114  519.468354
34       219             1140      108098  493.598174
35        45              223       22863  508.066667

In [21]:
order0.columns #列索引是复合索引，引用时要注意格式

MultiIndex([(         'info_id', 'count'),
            ('number_consumers',   'sum'),
            (     'expenditure',   'sum'),
            (     'expenditure',  'mean'),
            (            '人均消费',      '')],
           )

In [19]:
#按周统计人均消费信息,注意复合索引的用法
order0['人均消费']= order0[('expenditure','sum')]/order0[('number_consumers','sum')]
order0.round(2)

info_id number_consumers expenditure            人均消费
       count              sum         sum    mean        
week                                                     
31       223             1157      103218  462.86   89.21
32       212             1096      103185  486.72   94.15
33       237             1261      123114  519.47   97.63
34       219             1140      108098  493.60   94.82
35        45              223       22863  508.07  102.52

In [5]:
#按星期几统计
order['day_name']=order['lock_time'].dt.day_name()
order1=order.groupby(['day_name']).agg({"info_id":'count','number_consumers':'sum',"expenditure":['sum','mean']})
order1

info_id number_consumers expenditure            
            count              sum         sum        mean
day_name                                                  
Friday         75              395       35947  479.293333
Monday         87              461       39762  457.034483
Saturday      281             1527      138842  494.099644
Sunday        266             1358      140270  527.330827
Thursday       63              327       31671  502.714286
Tuesday        75              327       31918  425.573333
Wednesday      89              482       42068  472.674157

In [6]:
# 3.排序 
# 如果要将上面按星期统计的结果按照 周一，周二，这样的顺序排序，则应该怎么做？
# 尝试 1：在groupby中排序
order1=order.groupby(['day_name'],sort=False).agg({"info_id":'count','number_consumers':'sum',"expenditure":['sum','mean']})
order1

info_id number_consumers expenditure            
            count              sum         sum        mean
day_name                                                  
Monday         87              461       39762  457.034483
Tuesday        75              327       31918  425.573333
Wednesday      89              482       42068  472.674157
Thursday       63              327       31671  502.714286
Friday         75              395       35947  479.293333
Sunday        266             1358      140270  527.330827
Saturday      281             1527      138842  494.099644

In [7]:
# 尝试2： sort_values 排序
order1.sort_values(by='day_name', ascending=True, inplace=False)

#可以看出这个结果，不管怎么排，都是按照字符串顺序

info_id number_consumers expenditure            
            count              sum         sum        mean
day_name                                                  
Friday         75              395       35947  479.293333
Monday         87              461       39762  457.034483
Saturday      281             1527      138842  494.099644
Sunday        266             1358      140270  527.330827
Thursday       63              327       31671  502.714286
Tuesday        75              327       31918  425.573333
Wednesday      89              482       42068  472.674157

In [8]:
#尝试4，在order中添加进星期的序号  （可行）
order['daynum']=order['lock_time'].dt.weekday
order

,info_id,number_consumers,expenditure,lock_time,week,day_name,daynum
0,417,4,165,2016-08-01 11:11:46,31,Monday,0.0
1,301,3,321,2016-08-01 11:31:55,31,Monday,0.0
2,413,6,854,2016-08-01 12:54:37,31,Monday,0.0
3,415,4,466,2016-08-01 13:08:20,31,Monday,0.0
4,392,10,704,2016-08-01 13:07:16,31,Monday,0.0
...,...,...,...,...,...,...,...
940,641,8,679,2016-08-31 21:31:48,35,Wednesday,2.0
941,672,6,800,2016-08-31 21:56:12,35,Wednesday,2.0
942,692,8,735,2016-08-31 21:33:34,35,Wednesday,2.0
943,647,4,262,2016-08-31 21:55:39,35,Wednesday,2.0


In [9]:
#做同样的分组聚合，为了让星期的名称和序号对上，daynum应该取均值
order2=order.groupby(['day_name'],sort=False).agg({"info_id":'count','number_consumers':'sum',
                                                   "expenditure":['sum','mean'],
                                                  "daynum":'mean'})
order2

info_id number_consumers expenditure             daynum
            count              sum         sum        mean   mean
day_name                                                         
Monday         87              461       39762  457.034483    0.0
Tuesday        75              327       31918  425.573333    1.0
Wednesday      89              482       42068  472.674157    2.0
Thursday       63              327       31671  502.714286    3.0
Friday         75              395       35947  479.293333    4.0
Sunday        266             1358      140270  527.330827    6.0
Saturday      281             1527      138842  494.099644    5.0

In [11]:
order2.sort_values( by=('daynum','mean') )  #注意此时的排序字段为复合索引

info_id number_consumers expenditure             daynum
            count              sum         sum        mean   mean
day_name                                                         
Monday         87              461       39762  457.034483    0.0
Tuesday        75              327       31918  425.573333    1.0
Wednesday      89              482       42068  472.674157    2.0
Thursday       63              327       31671  502.714286    3.0
Friday         75              395       35947  479.293333    4.0
Saturday      281             1527      138842  494.099644    5.0
Sunday        266             1358      140270  527.330827    6.0

In [14]:
#隐去排序辅助列

order2.iloc[:,0:4].round(2)

info_id number_consumers expenditure        
            count              sum         sum    mean
day_name                                              
Monday         87              461       39762  457.03
Tuesday        75              327       31918  425.57
Wednesday      89              482       42068  472.67
Thursday       63              327       31671  502.71
Friday         75              395       35947  479.29
Sunday        266             1358      140270  527.33
Saturday      281             1527      138842  494.10

# 四. 数据框映射函数map、apply、applymap

### map：只用于Series，实现每个值->值的映射；

### apply：用于Series实现每个值的处理，用于Dataframe实现某个轴的Series的处理；

### applymap：只能用于DataFrame，用于处理该DataFrame的每个元素； 


In [40]:
import pandas as pd
%matplotlib inline
stocks=pd.read_excel('d:/aa/股票.xlsx')
stocks

,公司,日期,收盘,开盘,高,低,交易量,涨跌幅
0,BABA,2022-07-01,165.15,168.01,168.23,163.64,14.19,-0.0110
1,BABA,2022-07-02,165.77,162.82,166.88,161.90,11.60,0.0020
2,BABA,2022-07-03,169.48,166.65,170.18,165.00,10.39,0.0230
3,BIDU,2022-07-01,102.00,102.80,103.26,101.00,1.78,-0.0140
4,BIDU,2022-07-02,102.62,100.85,103.24,99.50,2.69,0.0150
5,BIDU,2022-07-03,104.32,102.35,104.73,101.15,2.24,0.0260
6,IQ,2022-07-01,15.92,16.14,16.22,15.50,11.65,-0.0170
7,IQ,2022-07-02,15.72,15.85,15.87,15.12,8.10,-0.0180
8,IQ,2022-07-03,16.06,15.71,16.38,15.32,10.08,0.0290
9,JD,2022-07-01,28.19,28.22,28.57,27.97,10.64,0.0010


###  Series.map(function) 或 Series.map(dict) 

In [41]:
# 1.增加涨跌列，根据涨跌幅，上涨股票为“涨”，下跌股票为“跌”
stocks['涨跌']=stocks['涨跌幅'].map(lambda x: "涨" if x>0 else '跌' )
stocks

,公司,日期,收盘,开盘,高,低,交易量,涨跌幅,涨跌
0,BABA,2022-07-01,165.15,168.01,168.23,163.64,14.19,-0.0110,跌
1,BABA,2022-07-02,165.77,162.82,166.88,161.90,11.60,0.0020,涨
2,BABA,2022-07-03,169.48,166.65,170.18,165.00,10.39,0.0230,涨
3,BIDU,2022-07-01,102.00,102.80,103.26,101.00,1.78,-0.0140,跌
4,BIDU,2022-07-02,102.62,100.85,103.24,99.50,2.69,0.0150,涨
5,BIDU,2022-07-03,104.32,102.35,104.73,101.15,2.24,0.0260,涨
6,IQ,2022-07-01,15.92,16.14,16.22,15.50,11.65,-0.0170,跌
7,IQ,2022-07-02,15.72,15.85,15.87,15.12,8.10,-0.0180,跌
8,IQ,2022-07-03,16.06,15.71,16.38,15.32,10.08,0.0290,涨
9,JD,2022-07-01,28.19,28.22,28.57,27.97,10.64,0.0010,涨


In [42]:
# 2. 增加公司的中文名称 ['阿里','百度','爱奇艺','京东']

stocks["公司"].unique()  #获取公司原名称

array(['BABA', 'BIDU', 'IQ', 'JD'], dtype=object)

In [43]:
# 制造字典：‘英文名’：‘中文名’

name_dic={ 'BABA':'阿里', 'BIDU':'百度', 'IQ':'爱奇艺', 'JD': '京东'    }
name_dic


{'BABA': '阿里', 'BIDU': '百度', 'IQ': '爱奇艺', 'JD': '京东'}

In [1]:
#利用字典，形成 英文名-中文名的映射



In [45]:
# 3. 按中文名称，日期 分组，计算交易量的平均值

stocks1=stocks.groupby('中文名称').agg({'交易量':'mean'}).round(2)
stocks1

,交易量
中文名称,
京东,9.65
爱奇艺,9.94
百度,2.24
阿里,12.06


In [2]:
# 4. 按照 1.阿里，2.百度，3.京东 4.爱奇艺  的顺序排序

# 构建映射字典的方式
seq_dict={'阿里':1,'百度':2,'京东':3,'爱奇艺':4}



In [62]:
# 4. 按照 1.阿里，2.百度，3.京东 4.爱奇艺  的顺序排序
# 方法二，类似字典法，构造一个映射数据框

sort_aid = pd.DataFrame(stocks['中文名称'].unique(),columns=['名称'])  # 制造排序辅助数据框
sort_aid['seq'] = [1,2,4,3]
sort_aid

,名称,seq
0,阿里,1
1,百度,2
2,爱奇艺,4
3,京东,3


In [64]:
# 将“中文名称”列改为索引
sort_aid = sort_aid.set_index('名称')  #此句不能掉，否则sort_aid没有发生改变
sort_aid

,seq
名称,
阿里,1
百度,2
爱奇艺,4
京东,3


In [72]:
stocks2=stocks.groupby('中文名称').agg({'交易量':'mean'}).round(2)
stocks2

,交易量
中文名称,
京东,9.65
爱奇艺,9.94
百度,2.24
阿里,12.06


In [73]:
# 两数组进行映射
stocks2['排序辅助'] = stocks2.index.map(sort_aid['seq'] )
stocks2

,交易量,排序辅助
中文名称,,
京东,9.65,3
爱奇艺,9.94,4
百度,2.24,2
阿里,12.06,1


In [74]:
#排序
stocks2.sort_values('排序辅助')

,交易量,排序辅助
中文名称,,
阿里,12.06,1
百度,2.24,2
京东,9.65,3
爱奇艺,9.94,4


### apply：用于Series实现每个值的处理，用于Dataframe实现某个轴的Series的处理；

In [75]:
stocks=pd.read_excel('d:/aa/股票.xlsx')
stocks

,公司,日期,收盘,开盘,高,低,交易量,涨跌幅
0,BABA,2022-07-01,165.15,168.01,168.23,163.64,14.19,-0.0110
1,BABA,2022-07-02,165.77,162.82,166.88,161.90,11.60,0.0020
2,BABA,2022-07-03,169.48,166.65,170.18,165.00,10.39,0.0230
3,BIDU,2022-07-01,102.00,102.80,103.26,101.00,1.78,-0.0140
4,BIDU,2022-07-02,102.62,100.85,103.24,99.50,2.69,0.0150
5,BIDU,2022-07-03,104.32,102.35,104.73,101.15,2.24,0.0260
6,IQ,2022-07-01,15.92,16.14,16.22,15.50,11.65,-0.0170
7,IQ,2022-07-02,15.72,15.85,15.87,15.12,8.10,-0.0180
8,IQ,2022-07-03,16.06,15.71,16.38,15.32,10.08,0.0290
9,JD,2022-07-01,28.19,28.22,28.57,27.97,10.64,0.0010


In [37]:
#为数据框添加两列，分别是 “开收盘差”， “高低差”
# ( 不明原因，此方法报错，分开则可以添加)

# def my_fun(x):
#     return (x['收盘']-x['开盘']) , (x['高']-x['低'])

# stocks[ ['开收盘差','高低差'] ] = stocks.apply( my_fun ,axis=1 ) 
# stocks

In [76]:
# #为数据框添加两列，分别是 “开收盘差”， “高低差” （分开添加列可以）

def my_fun(x):
    return (x['收盘']-x['开盘'])

stocks[ '开收盘差' ] = stocks.apply( my_fun , axis=1) 

stocks[ '高低差' ] = stocks.apply( lambda x: x['高']-x['低'] , axis=1) 

stocks[ '高低差1' ] = stocks['高']-stocks['低'] 

stocks


,公司,日期,收盘,开盘,高,低,交易量,涨跌幅,开收盘差,高低差,高低差1
0,BABA,2022-07-01,165.15,168.01,168.23,163.64,14.19,-0.0110,-2.86,4.59,4.59
1,BABA,2022-07-02,165.77,162.82,166.88,161.90,11.60,0.0020,2.95,4.98,4.98
2,BABA,2022-07-03,169.48,166.65,170.18,165.00,10.39,0.0230,2.83,5.18,5.18
3,BIDU,2022-07-01,102.00,102.80,103.26,101.00,1.78,-0.0140,-0.80,2.26,2.26
4,BIDU,2022-07-02,102.62,100.85,103.24,99.50,2.69,0.0150,1.77,3.74,3.74
5,BIDU,2022-07-03,104.32,102.35,104.73,101.15,2.24,0.0260,1.97,3.58,3.58
6,IQ,2022-07-01,15.92,16.14,16.22,15.50,11.65,-0.0170,-0.22,0.72,0.72
7,IQ,2022-07-02,15.72,15.85,15.87,15.12,8.10,-0.0180,-0.13,0.75,0.75
8,IQ,2022-07-03,16.06,15.71,16.38,15.32,10.08,0.0290,0.35,1.06,1.06
9,JD,2022-07-01,28.19,28.22,28.57,27.97,10.64,0.0010,-0.03,0.60,0.60


### applymap：只能用于DataFrame，用于处理该DataFrame的每个元素；

In [39]:
# #将数据框四列： ['收盘','开盘','高','低'] 变为整数

stocks[['收盘','开盘','高','低']].applymap(lambda x : int(x))

,收盘,开盘,高,低
0,165,168,168,163
1,165,162,166,161
2,169,166,170,165
3,102,102,103,101
4,102,100,103,99
5,104,102,104,101
6,15,16,16,15
7,15,15,15,15
8,16,15,16,15
9,28,28,28,27
